In [1]:
from typing import Any, Dict, List
import sys

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from transformers import pipeline, TextStreamer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain_core.outputs import LLMResult
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CHAT_MODEL_PATH = "/mnt/nfs/zsd_server/models/huggingface/chinese-alpaca-2-7b/"
CHAT_MODEL_PATH = "/mnt/nfs/zsd_server/models/huggingface/llama-3-chinese-8b-instruct-v3/"

In [3]:
# chat_model = AutoModelForCausalLM.from_pretrained(
#     CHAT_MODEL_PATH,
#     # load_in_8bit=True,
#     device_map='auto',
#     torch_dtype=torch.bfloat16,
# )
# tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL_PATH)

In [4]:
# streamer = TextStreamer(tokenizer)
# pipe = pipeline(
#     "text-generation",
#     model=chat_model,
#     tokenizer=tokenizer,
#     max_length=4096,
#     truncation=True,
#     repetition_penalty=1.2,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.95,
#     streamer=streamer,
#     return_full_text=False,
# )
# local_llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
class MyStreamingStdOutCallbackHandler(StreamingStdOutCallbackHandler):
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        sys.stdout.write("\n\n")
        sys.stdout.flush()

    # def on_chat_model_start(
    #     self,
    #     serialized: Dict[str, Any],
    #     messages: List[List[BaseMessage]],
    #     **kwargs: Any,
    # ) -> None:
    #     sys.stdout.write(f"{messages}\n\n")
    #     sys.stdout.flush()

In [6]:
callbacks = [MyStreamingStdOutCallbackHandler()]
local_llm = ChatOpenAI(
    model_name="llama-3-chinese-8b-instruct-v3-f16",
    openai_api_key="your-api-key",
    openai_api_base="http://localhost:11434/v1/",
    temperature=0.6,
    streaming=bool(callbacks),
    callbacks=callbacks,
)

In [7]:
text_contents = '''自2009年创立之初，阿里云就提出“云计算，
让计算成为公共服务”，并坚持通过云的弹性和自服务能力支持企业敏捷创新。
自2016年起，阿里云持续保持中国第一、亚太第一、全球第三的市场地位。
当前阿里云服务超400万客户，包括60%A股上市公司。
'''
fact_template = PromptTemplate(
    input_variables=["text_input"],
    template=("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>你是一个分析师。请从本文中提取关键事实。不要包括意见。给每个事实一个数字，并保持简短的句子。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: {text_input}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
"""),
)

fact_extraction_chain = LLMChain(
    llm=local_llm, prompt=fact_template, output_key="facts")

facts = fact_extraction_chain.run(text_contents)
facts

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


以下是从文章中提取的关键事实：

1. 阿里云在2009年成立，并提出将计算作为公共服务。
2. 阿里云一直致力于通过弹性和自服务能力支持企业创新。
3. 从2016年开始，阿里云保持中国、亚太地区第一和全球第三市场地位。
4. 目前，阿里云已拥有超过400万个客户，其中包括60%A股上市公司。

Error in MyStreamingStdOutCallbackHandler.on_llm_end callback: NameError("name 'sys' is not defined")


'以下是从文章中提取的关键事实：\n\n1. 阿里云在2009年成立，并提出将计算作为公共服务。\n2. 阿里云一直致力于通过弹性和自服务能力支持企业创新。\n3. 从2016年开始，阿里云保持中国、亚太地区第一和全球第三市场地位。\n4. 目前，阿里云已拥有超过400万个客户，其中包括60%A股上市公司。'

In [8]:
role_template = PromptTemplate(
    input_variables=["facts"],
    template=("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请根据以下事实清单，用中文为投资者写一个简短的段落。不要遗漏关键信息。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: {facts}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
"""),
)

data_analysis_chain = LLMChain(llm=local_llm, prompt=role_template, output_key="analysis")

data_analysis_facts = data_analysis_chain.run(facts)
data_analysis_facts

阿里云是一家成立于2009年的计算服务提供商，其主打产品是基于云的信息技术解决方案。自创建以来，它一直致力于支持企业创新，并推出弹性和自服务能力，以满足用户多样化的需求。此外，阿里云已经成功地巩固了在中国、亚太地区第一以及全球第三市场的地位。这是一个强大的优势，为公司赢得了广泛的认可。截至目前，阿里云已拥有超过400万个客户，其中包括60%的A股上市公司，这再次表明了它的卓越能力和价值在投资者心目中的重要地位。因此，我们有理由相信阿里云将继续保持其强大的市场竞争力，并为用户带来更先进、更可靠、高效率的计算服务解决方案。

Error in MyStreamingStdOutCallbackHandler.on_llm_end callback: NameError("name 'sys' is not defined")


'阿里云是一家成立于2009年的计算服务提供商，其主打产品是基于云的信息技术解决方案。自创建以来，它一直致力于支持企业创新，并推出弹性和自服务能力，以满足用户多样化的需求。此外，阿里云已经成功地巩固了在中国、亚太地区第一以及全球第三市场的地位。这是一个强大的优势，为公司赢得了广泛的认可。截至目前，阿里云已拥有超过400万个客户，其中包括60%的A股上市公司，这再次表明了它的卓越能力和价值在投资者心目中的重要地位。因此，我们有理由相信阿里云将继续保持其强大的市场竞争力，并为用户带来更先进、更可靠、高效率的计算服务解决方案。'

In [9]:
knowledge_template = PromptTemplate(
    input_variables=["facts"],
    template=("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请用中文，将以下事实列表转换为知识图谱的三元组。<|eot_id|>
<|start_header_id|>user<|end_header_id|>: {facts}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>:
""")
)

knowledge_chain = LLMChain(llm=local_llm, prompt=knowledge_template, output_key="knowledge")

knowledge_graph = knowledge_chain.run(facts)

以下是将事实列表转换为知识图谱三元组的结果：

（阿里云，成立于，2009年）
（阿里云，提出了，计算作为公共服务）

（阿里云，致力于，通过弹性和自服务能力支持企业创新）

（阿里云，保持市场地位，从，2016年开始）
（阿里云，保持中国第一，亚太地区第一）
（阿里云，保持全球第三）

（阿里云，有客户超过400万个）
（阿里云，包括60%，在A股上市公司之中）

Error in MyStreamingStdOutCallbackHandler.on_llm_end callback: NameError("name 'sys' is not defined")


In [10]:
# https://python.langchain.com.cn/docs/modules/chains/foundational/sequential_chains

In [11]:
simple_sequential_chain = SimpleSequentialChain(chains=[fact_extraction_chain, data_analysis_chain], verbose=True)
simple_sequential_chain

SimpleSequentialChain(verbose=True, chains=[LLMChain(prompt=PromptTemplate(input_variables=['text_input'], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>你是一个分析师。请从本文中提取关键事实。不要包括意见。给每个事实一个数字，并保持简短的句子。<|eot_id|>\n<|start_header_id|>user<|end_header_id|>: {text_input}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>:\n'), llm=ChatOpenAI(callbacks=[<__main__.MyStreamingStdOutCallbackHandler object at 0x729c00f76260>], client=<openai.resources.chat.completions.Completions object at 0x729c00f775e0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x729c00fb0d30>, model_name='llama-3-chinese-8b-instruct-v3-f16', temperature=0.6, openai_api_key=SecretStr('**********'), openai_api_base='http://localhost:11434/v1/', openai_proxy='', streaming=True), output_key='facts'), LLMChain(prompt=PromptTemplate(input_variables=['facts'], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>您是一名数据分析师。请根据以下事实清单，用中文为投资者写一个简短的段落。不要遗漏关键信息

In [12]:
simple_sequential_result = simple_sequential_chain.run(text_contents)
simple_sequential_result



> Entering new SimpleSequentialChain chain...
以下是提取自文本的关键事实：

1. 阿里云于2009年创立；
2. 阿里云提出的使命是将计算变为公共服务；
3. 阿里云坚持通过云的弹性和自助能力支持企业敏捷创新；
4. 自2016年以来，阿里云一直保持中国第一、亚太第一、全球第三的地位；
5. 目前，阿里云有超过400万个客户；
6. 其中，有60%的客户是A股上市公司。

Error in MyStreamingStdOutCallbackHandler.on_llm_end callback: NameError("name 'sys' is not defined")


以下是提取自文本的关键事实：

1. 阿里云于2009年创立；
2. 阿里云提出的使命是将计算变为公共服务；
3. 阿里云坚持通过云的弹性和自助能力支持企业敏捷创新；
4. 自2016年以来，阿里云一直保持中国第一、亚太第一、全球第三的地位；
5. 目前，阿里云有超过400万个客户；
6. 其中，有60%的客户是A股上市公司。
阿里云创立于2009年，旨在将计算变为公共服务，并推崇通过云的弹性和自助能力来支持企业敏捷创新。在过去的十余年中，阿里云始终保持着稳定的领导地位。截至目前，它已成为中国第一、亚太地区第一和全球第三大云计算提供商，其客户总数超过400万，其中包括了60%以上是来自A股上市公司的客户。这些客户能够充分利用阿里云的资源，以提高生产力和降低成本，并为未来的成功奠定坚实基础。无论是在创新或成长方面，阿里云一直秉承着将计算变革成为公共服务的使命，为全球企业提供卓越的云解决方案。

Error in MyStreamingStdOutCallbackHandler.on_llm_end callback: NameError("name 'sys' is not defined")


阿里云创立于2009年，旨在将计算变为公共服务，并推崇通过云的弹性和自助能力来支持企业敏捷创新。在过去的十余年中，阿里云始终保持着稳定的领导地位。截至目前，它已成为中国第一、亚太地区第一和全球第三大云计算提供商，其客户总数超过400万，其中包括了60%以上是来自A股上市公司的客户。这些客户能够充分利用阿里云的资源，以提高生产力和降低成本，并为未来的成功奠定坚实基础。无论是在创新或成长方面，阿里云一直秉承着将计算变革成为公共服务的使命，为全球企业提供卓越的云解决方案。

> Finished chain.


'阿里云创立于2009年，旨在将计算变为公共服务，并推崇通过云的弹性和自助能力来支持企业敏捷创新。在过去的十余年中，阿里云始终保持着稳定的领导地位。截至目前，它已成为中国第一、亚太地区第一和全球第三大云计算提供商，其客户总数超过400万，其中包括了60%以上是来自A股上市公司的客户。这些客户能够充分利用阿里云的资源，以提高生产力和降低成本，并为未来的成功奠定坚实基础。无论是在创新或成长方面，阿里云一直秉承着将计算变革成为公共服务的使命，为全球企业提供卓越的云解决方案。'

In [13]:
sequential_chain = SequentialChain(
    chains=[fact_extraction_chain, data_analysis_chain, knowledge_chain],
    input_variables=["text_input"],
    # Here we return multiple variables
    output_variables=["analysis", "knowledge"],
    verbose=True)

In [14]:
sequential_chain_result = sequential_chain({"text_input": text_contents})
sequential_chain_result



> Entering new SequentialChain chain...
以下

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


是从文中提取出来的关键事实：

1. 阿里云创立于2009年。
2. 阿里云提出「让计算成为公共服务」并推崇弹性和自服务能力支持企业创新。
3. 自2016年起，阿里云保持中国第一、亚太第一、全球第三的市场地位。
4. 阿里云当前有超过400万客户。
5. 其中包括60%的上市公司。

Error in MyStreamingStdOutCallbackHandler.on_llm_end callback: NameError("name 'sys' is not defined")


作为一名投资者，您可能对阿里云的发展和成就感兴趣。在2009年，阿里云正式成立，并提出了「让计算成为公共服务」的理念，强调弹性和自服务能力的重要性，以支持企业创新。如今，阿里云已经保持了多年的领导地位，在中国、亚太地区和全球范围内都稳居第一名。此外，截至目前，阿里云已拥有超过400万客户，其中包括60%以上的上市公司。这表明阿里云在行业中的巨大影响力，以及其能够为企业提供可靠、高效的解决方案。这些事实充分证明了阿里云的成长潜力和市场竞争力的强劲表现，对于投资者而言，这是一个值得关注、研究并加以分析的公司。

Error in MyStreamingStdOutCallbackHandler.on_llm_end callback: NameError("name 'sys' is not defined")


以下是将事实列表转换为知识图谱三元组的结果：

1. (阿里云，创立于，2009年)
2. (阿里云，提出了，「让计算成为公共服务」并推崇弹性和自服务能力支持企业创新)
3. ((阿里云，保持市场地位)，从2016年开始,(中国第一、亚太第一、全球第三))
4. ((阿里云，拥有客户数)，超过400万）
5. ((阿里云，拥有的客户种类),(上市公司),60%)

Error in MyStreamingStdOutCallbackHandler.on_llm_end callback: NameError("name 'sys' is not defined")



> Finished chain.


{'text_input': '自2009年创立之初，阿里云就提出“云计算，\n让计算成为公共服务”，并坚持通过云的弹性和自服务能力支持企业敏捷创新。\n自2016年起，阿里云持续保持中国第一、亚太第一、全球第三的市场地位。\n当前阿里云服务超400万客户，包括60%A股上市公司。\n',
 'analysis': '作为一名投资者，您可能对阿里云的发展和成就感兴趣。在2009年，阿里云正式成立，并提出了「让计算成为公共服务」的理念，强调弹性和自服务能力的重要性，以支持企业创新。如今，阿里云已经保持了多年的领导地位，在中国、亚太地区和全球范围内都稳居第一名。此外，截至目前，阿里云已拥有超过400万客户，其中包括60%以上的上市公司。这表明阿里云在行业中的巨大影响力，以及其能够为企业提供可靠、高效的解决方案。这些事实充分证明了阿里云的成长潜力和市场竞争力的强劲表现，对于投资者而言，这是一个值得关注、研究并加以分析的公司。',
 'knowledge': '以下是将事实列表转换为知识图谱三元组的结果：\n\n1. (阿里云，创立于，2009年)\n2. (阿里云，提出了，「让计算成为公共服务」并推崇弹性和自服务能力支持企业创新)\n3. ((阿里云，保持市场地位)，从2016年开始,(中国第一、亚太第一、全球第三))\n4. ((阿里云，拥有客户数)，超过400万）\n5. ((阿里云，拥有的客户种类),(上市公司),60%)'}